In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.svm import LinearSVC
from scipy.linalg import khatri_rao
from matplotlib import pyplot as plt
import time as t
import random

In [ ]:
y = np.array([1,0,0,1,0,1])
print(y)
y_new = 2*y
print(y_new)
y_last = (1-y_new)
print(y_last)

[1 0 0 1 0 1]
[2 0 0 2 0 2]
[-1  1  1 -1  1 -1]


In [ ]:
data = np.loadtxt( "/content/train.dat" )
data.shape

(10000, 9)

In [ ]:
y = data[:,-1]
print(y.shape)

(10000,)


In [ ]:
def createFeatures( X ):
    return np.cumprod( np.flip( 2 * X - 1 , axis = 1 ), axis = 1 )

data_trn, data_tst = train_test_split( data, train_size = 8000 )

X_raw = data_trn[:,:-1]
X = createFeatures( X_raw )
y = data_trn[:,-1]

y = np.where( y > 0, 1, -1 )

X_t_raw = data_tst[:,:-1]
X_t = createFeatures( X_t_raw )
y_t = data_tst[:,-1]

y_t = np.where( y_t > 0, 1, -1 )

clf = LinearSVC( loss = "hinge" )
clf.fit( X, y )
np.average( y_t == clf.predict( X_t ) )

0.671

In [ ]:
def createFeatures( X ):
    return np.cumprod( np.flip(1 - 2*X , axis = 1 ), axis = 1 )

In [ ]:
X_t = createFeatures(data[:,:-1])
Y_t = data[:,-1]
print(X_t.shape,Y_t.shape)

(10000, 8) (10000,)


In [ ]:
bias = np.array([1 for i in range(X_t.shape[0])])
print(bias.shape)

(10000,)


In [ ]:
X_t = np.c_[X_t,bias]
print(X_t.shape)
print(X_t[0])

(10000, 9)
[ 1.  1. -1. -1. -1. -1. -1. -1.  1.]


In [ ]:
fin_Xt = khatri_rao(X_t.T,X_t.T)
fin_Xt = khatri_rao(fin_Xt,X_t.T)
print(fin_Xt.shape)

(729, 10000)


In [ ]:
def get_renamed_labels(y):
  y_new = 2*y
  y_new = (1-y_new)
  return y_new

In [ ]:
fin_Yt = get_renamed_labels(Y_t)
print(Y_t)
print(fin_Yt)

[1. 0. 1. ... 1. 1. 1.]
[-1.  1. -1. ... -1. -1. -1.]


In [ ]:
print(fin_Xt.shape,fin_Yt.shape)

(729, 10000) (10000,)


In [ ]:
n = 500
X = fin_Xt.T
y = fin_Yt
C = 1
neta = 1
epochs = 10
print(X.shape)
print(y.shape)

(10000, 729)
(10000,)


In [ ]:
g = np.zeros( (9,) )
print(g.shape)
print(g)
g[5 < 1] = -1
print(g)

(9,)
[0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
def getCSVMObjVal( theta ):
    w = theta[0:-1]
    b = theta[-1]
    hingeLoss = np.maximum( 1 - np.multiply( (X.dot( w ) + b), y ), 0 )
    return 0.5 * w.dot( w ) + C * np.sum( hingeLoss )

In [ ]:
theta = X
w = theta[0:-1]
b = theta[-1]
print(w.shape)
print(b.shape)

(9999, 729)
(729,)


In [ ]:
def getCSVMGrad( theta,w ):
    discriminant = np.multiply( (X.dot( w )), y )
    g = np.zeros( (y.size,) )
    g[discriminant < 1] = -1
    #delb = C * g.dot( y )
    delw = w + C * (X.T * g).dot( y )
    return np.array(delw)

In [ ]:
w = np.ones((X.shape[1],))
print(w.shape)

(729,)


In [ ]:
delw = np.array((getCSVMGrad(X,w)))
print(delw.shape)

(729,)


In [ ]:
from tqdm import tqdm

In [ ]:
epochs = 1000
C = 1
neta = 0.001

In [ ]:
def applySVM(X,w):
  return X.dot(w)

In [ ]:
X_trn = np.flip(createFeatures(data_trn[:,:-1]))
bias = np.array([1 for i in range(X_trn.shape[0])])
Y_trn = data_trn[:,-1]
X_trn = np.c_[X_trn,bias]
Y_trn = get_renamed_labels(Y_trn)
fin_Xt = khatri_rao(X_trn.T,X_trn.T)
X_trn = khatri_rao(fin_Xt,X_trn.T)
X_trn = X_trn.T
print(X_trn.shape,Y_trn.shape)

(8000, 729) (8000,)


In [ ]:
w = np.zeros((X.shape[1],))
maxx = 0
epoch = 0
for i in tqdm(range(epochs)):
  delw = getCSVMGrad(X,w)
  w = w - neta*(delw)
  y_t_pred = applySVM(X,w)
  y_t_pred = np.where( y_t_pred > 0, 1, -1 )
  av = np.average( y == y_t_pred )
  print(av)
  if av > maxx:
    epoch = i
    maxx = av
print(maxx)

  0%|          | 3/1000 [00:00<00:40, 24.84it/s]

0.8056
0.8526
0.8862
0.8946


  1%|          | 6/1000 [00:00<00:37, 26.17it/s]

0.8975
0.8898


  1%|          | 9/1000 [00:00<00:37, 26.77it/s]

...

1.0
1.0
1.0
1.0
1.0


 99%|█████████▉| 992/1000 [00:44<00:00, 25.19it/s]

1.0
1.0
1.0
1.0
1.0
1.0


100%|██████████| 1000/1000 [00:45<00:00, 22.18it/s]

1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [ ]:
print(maxx,epoch)

1.0 121


In [ ]:
#test data
test_data = np.loadtxt( "/content/test.dat" )
test_data.shape

(20000, 9)

In [ ]:
X_test = np.flip(createFeatures(data[:,:-1]))
bias = np.array([1 for i in range(X_test.shape[0])])
Y_test = data[:,-1]
X_test = np.c_[X_test,bias]
Y_test = get_renamed_labels(Y_test)
fin_Xt = khatri_rao(X_test.T,X_test.T)
X_test = khatri_rao(fin_Xt,X_test.T)
X_test = X_test.T
print(X_test.shape,Y_test.shape)

(10000, 729) (10000,)


In [ ]:
new_w = np.zeros((X.shape[1],))
maxx = 0
epoch = 0
for i in tqdm(range(epochs)):
  delw = getCSVMGrad(X,new_w)
  new_w = w - neta*(delw)
  y_t_pred = applySVM(X,new_w)
  y_t_pred = np.where( y_t_pred > 0, 1, -1 )
  av = np.average( y == y_t_pred )
  w=new_w
  print(av)
  if av > maxx:
    epoch = i
    maxx = av
print(maxx)

  0%|          | 3/1000 [00:00<00:43, 22.94it/s]

0.8669
0.9177
0.9426
0.9489
0.9523


  1%|          | 9/1000 [00:00<00:38, 25.43it/s]

0.9387
0.941
0.9443
0.953
0.9696
0.9635


  2%|▏         | 15/1000 [00:00<00:37, 26.20it/s]

...

1.0
1.0
1.0
1.0
1.0
1.0


100%|█████████▉| 996/1000 [00:38<00:00, 26.56it/s]

1.0
1.0
1.0
1.0
1.0
1.0


100%|██████████| 1000/1000 [00:38<00:00, 25.99it/s]

1.0
1.0
1.0
1.0
1.0
